In [1]:
# This is a code to generate simple input files for Unstructured grid MODFLOW 6 groundwater flow model
# that has ul
import os
import numpy as np
import pandas as pd
import re

C:\Users\rhasan1\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
rgn = int(1)

In [3]:
# current directory
current_dir = os.getcwd()

# Sub models path
subModelPath = os.path.join(current_dir,f"Region_{rgn}")

# save new files
toSavePath = os.path.join(subModelPath,"Packages")

regionName = f"Region{rgn}"

In [4]:
### Read DISU 
disu_path = os.path.join(subModelPath,f"Region{rgn}.disu")

with open(disu_path, 'r') as file:
    disuFile = file.read() 

In [5]:
#### Get Dimensions FROM DISU

def extract_dimensions_as_dict(file_contents):
    # Pattern of DISU Dimensions block
    pattern = r"BEGIN DIMENSIONS\n(.*?)END DIMENSIONS"
    
    # Search above pattern 
    match = re.search(pattern, file_contents, re.DOTALL)
    
    if match:
        # Extract 
        dimensions_content = match.group(1).strip()

        # Save each info in a dict
        dimensions_dict = {}
        for line in dimensions_content.split('\n'):
            parts = line.split()
            if len(parts) == 2:
                key, value = parts
                dimensions_dict[key] = int(value)  # In Dimensions block all values are integer number

        return dimensions_dict
    else:
        return "DIMENSIONS section not found"

# DISU Dimension
dimensions_dict = extract_dimensions_as_dict(disuFile)

DISU_nodes=dimensions_dict["NODES"]
DISU_NJA=dimensions_dict["NJA"]
DISU_NVERT=dimensions_dict["NVERT"]

print(DISU_nodes, DISU_NJA, DISU_NVERT)

607550 5964336 35911


In [6]:
############ X-Y Coordinate of each nodes (3D cells) ==> FROM DISU 


def extract_vertices_as_dict(file_contents):
    pattern = r"BEGIN CELL2D\n(.*?)END CELL2D"
    match = re.search(pattern, file_contents, re.DOTALL)
    
    if match:
        nodes_xy_content = match.group(1).strip()
        cellId_List=[]
        X_List=[]
        Y_List=[]

        # Split the content into lines
        for line in nodes_xy_content.split('\n'):
            parts = line.split() # there are 3 parts in each line. cellid, x, y
            if len(parts)> 3:
                # First value as cellid, next two values are X and Y
                cellId = int(parts[0])
                X = float(parts[1])
                Y = float(parts[2])
                cellId_List.append(cellId)
                X_List.append(X)
                Y_List.append(Y)

        return [cellId_List, X_List, Y_List]
    else:
        return "NODES X-Y coordinate value not found"

#Read 3d-cells x and y
R_3D_nodes_XY_dict = extract_vertices_as_dict(disuFile)

print(len(R_3D_nodes_XY_dict[0]))

607550


In [15]:
##### read TOP of each 3D cell

r_TOP_path=os.path.join(subModelPath,"top_mf6.dat")

with open(r_TOP_path, 'r') as file:
    r_tops = file.read().split()


##### read Bottom of each 3D cell

r_BOT_path=os.path.join(subModelPath,"bot_mf6.dat")

with open(r_BOT_path, 'r') as file:
    r_bots = file.read().split()

In [16]:
# this function create a dataframe for storing values for basic package
def create_dataframe(n):
    df = pd.DataFrame({
        'cellid': range(1, n + 1),
        'cellTop':r_tops,
        "cellBot":r_bots,
        'IC': [0] * n, # Dummy value to start the model
        'K' : [10] * n, # Dummy value to start the model
        'K33' :[1] * n, # Dummy value to start the model
        'SS' :[0.000001] * n, # Dummy value to start the model
        'SY' :[0.001] * n # Dummy value to start the model
    })
    return df

In [17]:
df_basic = create_dataframe(DISU_nodes) # Number of Nodes means number of 3D cells
df_working = df_basic[list(df_basic.columns.values)]

# Add lat and long
df_working['X'] = R_3D_nodes_XY_dict[1]
df_working['Y'] = R_3D_nodes_XY_dict[2]

In [18]:
df_working

,cellid,cellTop,cellBot,IC,K,K33,SS,SY,X,Y
0,1,31.657700,30.157700,0,10,1,0.000001,0.001,439455.0,1114785.0
1,2,30.157700,22.420000,0,10,1,0.000001,0.001,439455.0,1114785.0
2,3,22.420000,-3.738600,0,10,1,0.000001,0.001,439455.0,1114785.0
3,4,-3.738600,-99.462608,0,10,1,0.000001,0.001,439455.0,1114785.0
4,5,-99.462608,-150.929993,0,10,1,0.000001,0.001,439455.0,1114785.0
...,...,...,...,...,...,...,...,...,...,...
607545,607546,-503.940002,-505.140015,0,10,1,0.000001,0.001,310455.0,939785.0
607546,607547,-505.140015,-542.340027,0,10,1,0.000001,0.001,310455.0,939785.0
607547,607548,-542.340027,-544.140015,0,10,1,0.000001,0.001,310455.0,939785.0
607548,607549,-544.140015,-547.440002,0,10,1,0.000001,0.001,310455.0,939785.0


In [13]:
# Saving the daraframe with x,y coordinates to have 2D vizualization (of head, concentration, etc.) in ArcPro in future

# csv filename
csv_filename = toSavePath + "\\" +regionName + ".csv"
df_working.to_csv(f'{csv_filename}')

In [10]:
# Template for NPF file
template = """# NPF: Node Property Flow package file created on 11/1/2024 by Rafin Hasan
BEGIN OPTIONS
#  SAVE_FLOWS
#  SAVE_SPECIFIC_DISCHARGE
#  SAVE_SATURATION
END OPTIONS

BEGIN GRIDDATA
  ICELLTYPE
    CONSTANT 0
  K
    INTERNAL FACTOR  1.0
{K_values}
  K33
    INTERNAL FACTOR  1.0
{K33_values}
END GRIDDATA"""

# Extracting 'K' and 'K33' 
K_values = '\n'.join(['\t\t' + str(k) for k in df_working['K']])
K33_values = '\n'.join(['\t\t' + str(k33) for k33 in df_working['K33']])

# Updating template
formatted_content = template.format(K_values=K_values, K33_values=K33_values)

# Filename
filename = toSavePath + "\\" +regionName + ".npf"

# Writing NPF
with open(filename, 'w') as file:
    file.write(formatted_content)

print(f"File '{filename}' has been created.")

File 'C:\Users\rhasan1\Documents\Summer-24\0.NewMsThesis\MERAS_GeneratePackages_New\Region_1\Packages\Region1.npf' has been created.


In [11]:
# Template for IC
template = """# IC: Initial Conditions package file created on 11/1/2024 by Rafin Hasan
BEGIN OPTIONS
END OPTIONS

BEGIN GRIDDATA
    STRT
    INTERNAL FACTOR 1.0
{STRT_values}
END GRIDDATA"""

# Extracting IC
STRT_values = '\n'.join(['\t\t' + str(strt) for strt in df_working['IC']])

# Updating template
formatted_content = template.format(STRT_values=STRT_values)

# Filename
filename = toSavePath + "\\" +regionName + ".ic"

# Writing IC
with open(filename, 'w') as file:
    file.write(formatted_content)

print(f"File '{filename}' has been created.")

File 'C:\Users\rhasan1\Documents\Summer-24\0.NewMsThesis\MERAS_GeneratePackages_New\Region_1\Packages\Region1.ic' has been created.


In [12]:
# Template for STO
template = """# STO: Storage package file created on 11/1/2024 by Rafin Hasan
BEGIN OPTIONS
    SAVE_FLOWS
END OPTIONS

BEGIN GRIDDATA
    ICONVERT
        constant 0
    SS
    INTERNAL FACTOR 1.0
{SS_values}
    SY
    INTERNAL FACTOR 1.0
{SY_values}
END GRIDDATA

BEGIN PERIOD 1
    TRANSIENT
END PERIOD"""

# Extracting 'SS' and 'SY'
SS_values = '\n'.join(['\t\t' + str(ss) for ss in df_working['SS']])
SY_values = '\n'.join(['\t\t' + str(sy) for sy in df_working['SY']])

# Updating template
formatted_content = template.format(SS_values=SS_values, SY_values=SY_values)

# Filename
filename = toSavePath + "\\" +regionName + ".sto"

# Writing STO
with open(filename, 'w') as file:
    file.write(formatted_content)

print(f"File '{filename}' has been created.")

File 'C:\Users\rhasan1\Documents\Summer-24\0.NewMsThesis\MERAS_GeneratePackages_New\Region_1\Packages\Region1.sto' has been created.
